In [33]:
## The neuropath data at Emory ADRC is currently not particularly well organized
## and naming schema is inconsistent.  Working on developing a schema 
import girder_client
import os,sys, json
import dagSecrets as dgs
from collections import Counter
import pandas as pd
gc = girder_client.GirderClient(apiUrl=dgs.apiUrl)
gc.authenticate(apiKey=dgs.apiToken)

{'_id': '5f3d80d2615f2a24eb8fee20'}

In [3]:
## Get the ID for the ADRC Collection
rootCollectionName = 'Emory_ADRC'
rawColList = gc.get('collection?text=%s' % rootCollectionName)
adrcCollection = [x for x in rawColList if x['name']==rootCollectionName][0]

In [8]:
## Go through each year... note I am adding code to ignore folders that are not numeric
npSlideDict = {}


for yr in gc.listFolder(adrcCollection['_id'],parentFolderType='collection'):
    year = yr['name']
    if not year.isnumeric():
        continue
    else:
        print(year)
    for subj in gc.listFolder(yr['_id']):
        ### To do.. generate number of slides under each subject
        #### And in the future also validate the metadata!!
        subjItemSet = gc.listResource('resource/%s/items?type=folder' % subj['_id'])
        slideList = [x for x in subjItemSet if (x['name'].endswith('.ndpi') or x['name'].endswith('svs'))]
        
        print("\t%s\t%s" % (subj['name'],len(slideList)))
        npSlideDict["%s_%s" % (year,subj['name'])] = slideList

1986
	E36-04	3
1989
	E39-44	8
1994
	OS94-54	67
1996
	E96-33	118
1997
	E97-53	77
	E97-60	20
	E97-61	3
1998
	OS98-11	57
1999
	E99-04	18
	E99-07	70
	E99-26	96
	E99-30	98
2000
	E00-06	60
	E00-10	58
	E00-11	53
	E00-12	50
	E00-14	49
	E00-18	92
	E00-19	19
	E00-21	16
	E00-28	57
	E00-32	103
	E00-40	59
	E00-41	3
2001
	E01-02	1
	E01-06	61
	E01-10	2
	E01-134	11
	E01-152	7
	E01-33	16
	E01-60	10
	E01-70	51
	E01-94	7
2002
	E02-207	37
	E02-208	65
	E02-35	53
	E02-66	36
2003
	E03-138	8
	E03-163	55
	E03-299	36
	E03-300	1
	E03-385	7
	E03-392	71
	E03-395	34
	E03-396	29
	E03-55	11
	E03-95	13
2004
	E04-125	56
	E04-139	106
	E04-152	79
	E04-155	25
	E04-169	66
	E04-186	74
	E04-48	1
	E04-49	71
	E04-68	32
	E04-76	11
	E04-99	13
2005
	E05-04	77
	E05-100	1
	E05-108	20
	E05-130	67
	E05-132	69
	E05-179	66
	E05-194	82
	E05-200	14
	E05-56	74
	E05-74	40
	E05-81	92
	E05-87	52
2006
	E06-06	128
	E06-112	21
	E06-128	69
	E06-13	67
	E06-15	69
	E06-155	73
	E06-160	22
	E06-18	78
	E06-41	68
	E06-52	10
	E06-97	113
2007
	E07-48	23


In [31]:
## meta.region  meta.stain  meta.subject
basicSlideInfo = []
mdKeys = ['region','stain','subject','brain_region']

for subjyr in npSlideDict:
    for s in npSlideDict[subjyr]:
        slideData = {'folderId': subjyr,'slideName':s['name'],'itemId':s['_id']}
        
        for k in mdKeys:
            if k in s['meta']:
                slideData[k] = s['meta'][k]
        basicSlideInfo.append(slideData)

In [35]:
pd.DataFrame(basicSlideInfo).to_csv("adrcInitialMetadataExport.csv")